In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_br.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date   value    liter
0 2017-12-31  255.06   412770
1 2018-12-31  259.33   720236
2 2019-12-31  266.04  1330163
3 2020-12-31  267.16  2430153
4 2021-12-31  264.94  3296386
5 2022-12-31  291.40  4141401
6 2023-12-31  301.80  4419242


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7 non-null      datetime64[ns]
 1   value   7 non-null      float64       
 2   liter   7 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 296.0 bytes


In [4]:
df.drop(['liter'], axis = 1,inplace=True)

In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

             value
date              
2017-12-31  255.06
2018-12-31  259.33
2019-12-31  266.04
2020-12-31  267.16
2021-12-31  264.94
2022-12-31  291.40
2023-12-31  301.80


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. 

When integer is passed it means N continuous points in the future without any gap. 

If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [10]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
naive,Naive Forecaster,2.7152,2.3428,9.9333,9.9333,0.0345,0.0359,5.7700
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.7509,2.3972,10.5779,10.5779,0.0376,0.0382,0.5733
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,2.7525,2.3937,10.4923,10.4923,0.0372,0.0379,0.1600
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383,0.0900
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383,0.1333
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,2.7649,2.4054,10.5806,10.5806,0.0376,0.0382,0.0900
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,2.7925,2.4334,10.7459,10.7459,0.0382,0.0388,0.4167
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.7965,2.4332,10.7201,10.7201,0.0382,0.0387,0.0867
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.7967,2.4334,10.7208,10.7208,0.0382,0.0387,0.0767
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,2.7967,2.4334,10.7208,10.7208,0.0382,0.0387,0.3900


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## llar_cds_dt


In [18]:
arima = exp.create_model('llar_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.6849,0.6686,3.7600,3.7600,0.0141,0.0140
1,2020,1.9631,1.7075,7.9180,7.9180,0.0299,0.0294
2,2021,5.6243,4.8326,20.1350,20.1350,0.0691,0.0716
Mean,NaT,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383
SD,NaT,2.0933,1.7696,6.9497,6.9497,0.0231,0.0243



## ada_cds_dt


In [19]:
theta = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:57:08
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,AdaBoost w/ Cond. Deseasonalize & Detrending


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.5738,0.5601,3.1500,3.1500,0.0118,0.0117
1,2020,1.8082,1.5727,7.2930,7.2930,0.0275,0.0272
2,2021,5.8754,5.0483,21.0340,21.0340,0.0722,0.0749
Mean,NaT,2.7525,2.3937,10.4923,10.4923,0.0372,0.0379
SD,NaT,2.2650,1.9221,7.6436,7.6436,0.0256,0.0269


## lightgbm_cds_dt

In [20]:

ets_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:57:15
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting w/ Cond. Deseasonalize...


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.6849,0.6686,3.7600,3.7600,0.0141,0.0140
1,2020,1.9435,1.6904,7.8387,7.8387,0.0296,0.0292
2,2021,5.6243,4.8326,20.1350,20.1350,0.0691,0.0716
Mean,NaT,2.7509,2.3972,10.5779,10.5779,0.0376,0.0382
SD,NaT,2.0958,1.7719,6.9600,6.9600,0.0232,0.0244


## naive

In [21]:

ets_dt = exp.create_model('naive')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.2040,0.1991,1.1200,1.1200,0.0042,0.0042
1,2020,0.5504,0.4787,2.2200,2.2200,0.0084,0.0083
2,2021,7.3911,6.3506,26.4600,26.4600,0.0908,0.0951
Mean,NaT,2.7152,2.3428,9.9333,9.9333,0.0345,0.0359
SD,NaT,3.3094,2.8362,11.6947,11.6947,0.0399,0.0419


In [22]:
## LITER

In [23]:
#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_mt.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date  value    liter
0 2017-12-31  190.2   301961
1 2018-12-31  192.9   545110
2 2019-12-31  194.8  1029190
3 2020-12-31  197.3  1997490
4 2021-12-31  200.8  2860097
5 2022-12-31  204.7  3209840
6 2023-12-31  208.2  4419242


In [24]:
df.drop(['value'], axis = 1,inplace=True)

In [25]:
df.set_index('date',inplace=True)

In [26]:
df=df.sort_index(axis=0)
print(df.head(10))

              liter
date               
2017-12-31   301961
2018-12-31   545110
2019-12-31  1029190
2020-12-31  1997490
2021-12-31  2860097
2022-12-31  3209840
2023-12-31  4419242


In [27]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,liter
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [28]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [29]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [30]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
theta,Theta Forecaster,0.8966,0.8327,406542.6589,406542.6589,0.1688,0.1875,0.0233
polytrend,Polynomial Trend Forecaster,0.9415,0.8718,417119.4556,417119.4556,0.1769,0.2030,1.8233
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,0.9535,0.8789,428580.5333,428580.5333,0.1798,0.2073,0.0667
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.9581,0.8867,420193.3110,420193.3110,0.1799,0.2090,0.1167
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.0139,0.9396,440596.6227,440596.6227,0.1900,0.2238,0.0367
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.0417,0.9625,461486.2807,461486.2807,0.1959,0.2301,2.6933
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,1.0639,0.9837,468735.7000,468735.7000,0.1999,0.2363,2.3600
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.0639,0.9837,468735.6764,468735.6764,0.1999,0.2363,0.3000
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.0639,0.9837,468733.6971,468733.6971,0.1999,0.2363,0.1033
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0681,0.9876,471440.8787,471440.8787,0.2008,0.2371,2.3400


## lightgbm_cds_dt

In [31]:
arima = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.8286,1.7359,664918.2507,664918.2507,0.3329,0.3993
1,2020,0.9826,0.8670,555366.9323,555366.9323,0.1942,0.2151
2,2021,0.0630,0.0573,40294.7500,40294.7500,0.0126,0.0125
Mean,NaT,0.9581,0.8867,420193.3110,420193.3110,0.1799,0.2090
SD,NaT,0.7210,0.6854,272326.4711,272326.4711,0.1312,0.1580


## ADA

In [32]:
arima = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.6630,1.5786,604685.5000,604685.5000,0.3027,0.3567
1,2020,1.1391,1.0050,643786.9000,643786.9000,0.2251,0.2536
2,2021,0.0583,0.0530,37269.2000,37269.2000,0.0116,0.0117
Mean,NaT,0.9535,0.8789,428580.5333,428580.5333,0.1798,0.2073
SD,NaT,0.6681,0.6292,277158.9791,277158.9791,0.1231,0.1446


## POLYTREND


In [33]:
arima = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.7734,1.6834,644840.6667,644840.6667,0.3228,0.3850
1,2020,0.8829,0.7790,498992.5000,498992.5000,0.1745,0.1911
2,2021,0.1681,0.1530,107525.2000,107525.2000,0.0335,0.0329
Mean,NaT,0.9415,0.8718,417119.4556,417119.4556,0.1769,0.2030
SD,NaT,0.6567,0.6282,226869.0874,226869.0874,0.1181,0.1440


## THETA

In [34]:
arima = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.6671,1.5826,606198.3973,606198.3973,0.3035,0.3578
1,2020,0.5467,0.4824,308994.2675,308994.2675,0.1080,0.1142
2,2021,0.4760,0.4332,304435.3119,304435.3119,0.0948,0.0906
Mean,NaT,0.8966,0.8327,406542.6589,406542.6589,0.1688,0.1875
SD,NaT,0.5456,0.5306,141190.1942,141190.1942,0.0954,0.1208


## XGBOOST

In [35]:
arima = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})



,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9943,1.8931,725150.9792,725150.9792,0.3630,0.4435
1,2020,1.1391,1.0050,643786.8906,643786.8906,0.2251,0.2536
2,2021,0.0583,0.0530,37269.1594,37269.1594,0.0116,0.0117
Mean,NaT,1.0639,0.9837,468735.6764,468735.6764,0.1999,0.2363
SD,NaT,0.7922,0.7514,306895.7969,306895.7969,0.1446,0.1767
